This notebook is related to the conference proceedings paper of LeVeque & Yong.  At the end of the notebook we look at the 
stegoton solution values compared to the Riemann invariants of the first-order homogenized system.

In [ ]:
from clawpack import pyclaw
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation
import numpy as np
from stegoton import *
from IPython.display import HTML
from scipy.integrate import quad

font = {'size'   : 15}
matplotlib.rc('font', **font)
ifft = np.fft.ifft

In [ ]:
def stress(frame):
    """Compute stress from strain and momentum"""
    from clawpack.riemann.nonlinear_elasticity_1D_py import sigma 
    aux = frame.aux
    epsilon = frame.q[0,:]
    stress = sigma(frame.q,aux,{'stress_relation':'exponential'})
    return stress

# Set up and run

In [ ]:
tfinal = 200.
A = 1.0
cpl = 160

claw = setup(ic=2, num_output_times=400, tfinal=tfinal, cells_per_layer=cpl, amp=A, solver_type="sharpclaw")
claw.solution.state.grid.add_gauges([(150.25,)])
claw.solution.state.grid.add_gauges([(150.75,)])
claw.solver.cfl_max = 1.9
claw.solver.cfl_desired = 1.85

claw.run()

In [ ]:
import matplotlib
from matplotlib import animation
matplotlib.rcParams['animation.embed_limit'] = 2**128
from IPython.display import HTML

fig, ax = plt.subplots(figsize=[12,8], nrows=2, ncols=1, sharex=True)
fig.subplots_adjust(hspace=0)

#fig = plt.figure(figsize=[10,6])
#ax = plt.axes(xlim=(xc[0], xc[-1]), ylim=(2.4, 2.9))
ax[0].set_ylim(-0.02,0.4)
ax[1].set_ylim(-0.02,1.4)
line, = ax[0].plot([], [], lw=2)
line2, = ax[1].plot([], [], lw=2)

xc = claw.frames[0].grid.x.centers

def fplot(i):
    t = i*claw.tfinal/claw.num_output_times
    tx = 1.*t + 20
    xl = max(tx-60,0)
    xr = max(tx, 60)

    frame = claw.frames[i]
    eps = frame.q[0,:]
    rho = frame.aux[0,:]
    K = frame.aux[1,:]
    sigma = np.exp(K*eps)-1
    q2 = frame.q[1,:]
    line.set_data(xc, -q2/rho)
    line2.set_data(xc, sigma)
    ax[0].set_title("t= {:.1f}".format(frame.t))
    ax[0].set_ylabel('-u',fontsize=20)
    ax[1].set_ylabel('Stress',fontsize=20)
    ax[0].set_xlim(xl,xr)
    ax[1].set_xlim(xl,xr)
    
    fname = 'frame'+str(i).zfill(4)+'.png'
    fig.savefig(fname)
    
    return line,

anim = animation.FuncAnimation(fig, fplot, frames=401, repeat=False)
plt.close()
HTML(anim.to_jshtml())

In [ ]:
anim.save("stegoton_train_follow.mp4")

# Gauges at layer midpoints

In [ ]:
gaugeA = np.loadtxt('./_output/_gauges/gauge150.25.txt')
tA = gaugeA[:,0]
strainA = gaugeA[:,1]
rhouA = gaugeA[:,2]
rhoA = 1.; KA = 1.

sigA = np.exp(KA*strainA) - 1.
uA = rhouA / rhoA

In [ ]:
gaugeB = np.loadtxt('./_output/_gauges/gauge150.75.txt')
tB = gaugeB[:,0]
strainB = gaugeB[:,1]
rhouB = gaugeB[:,2]
rhoB = 4.; KB = 4.

sigB = np.exp(KB*strainB) - 1.
uB = rhouB / rhoB

In [ ]:
plt.plot(sigA, uA, '-b')
plt.plot(sigB, uB, '-r')
ss = np.linspace(0,1)
uu = 1-np.sqrt(ss+1)
plt.plot(ss, uu, '-k')
plt.xlabel(r'$\sigma$',fontsize=20)
plt.ylabel(r'$u$',fontsize=20);

In [ ]:
plt.plot(tA,sigA)
plt.xlim(150,200)

In [ ]:
i = 20
frame = claw.frames[i]
q = frame.q
xc = frame.grid.x.centers
rho = frame.aux[0,:]
u = q[1,:]/rho
sig = stress(frame)

Kavg = 8/5.
rhoavg = 5/2

sigavg = np.exp(Kavg*q[0,:]) - 1.
uB = q[1,:] / rhoavg

In [ ]:
plt.plot(sig, u)
ss = np.linspace(0,1)
uu = 1-np.sqrt(ss+1)
plt.plot(ss, uu, '-k')

In [ ]:
sig_window = np.lib.stride_tricks.sliding_window_view(sig, cpl)
u_window = np.lib.stride_tricks.sliding_window_view(u, cpl)

sig_avg = sig_window.mean(1)
u_avg = u_window.mean(1)

In [ ]:
plt.plot(sig_avg,u_avg)
ss = np.linspace(0,1)
uu = 1-np.sqrt(ss+1)
plt.plot(ss, uu, '-k')
plt.plot(ss,-uu, '-k');
plt.xlabel(r'$\sigma$',fontsize=20)
plt.ylabel(r'$u$',fontsize=20);

# Animation of both wave trains

In [ ]:
import matplotlib
from matplotlib import animation
matplotlib.rcParams['animation.embed_limit'] = 2**128
from IPython.display import HTML

fig, ax = plt.subplots(figsize=[12,8], nrows=2, ncols=1, sharex=True)
fig.subplots_adjust(hspace=0)

#fig = plt.figure(figsize=[10,6])
#ax = plt.axes(xlim=(xc[0], xc[-1]), ylim=(2.4, 2.9))
ax[0].set_ylim(-0.4,0.4)
ax[1].set_ylim(-0.02,1.6)
line, = ax[0].plot([], [], lw=2)
line2, = ax[1].plot([], [], lw=2)

xc = claw.frames[0].grid.x.centers

def fplot(i):
    t = i*claw.tfinal/claw.num_output_times

    frame = claw.frames[i]
    eps = frame.q[0,:]
    rho = frame.aux[0,:]
    K = frame.aux[1,:]
    sigma = np.exp(K*eps)-1
    q2 = frame.q[1,:]
    line.set_data(xc, -q2/rho)
    line2.set_data(xc, sigma)
    ax[0].set_title("t= {:.1f}".format(frame.t))
    ax[0].set_ylabel('u',fontsize=20)
    ax[1].set_ylabel(r'$\sigma$',fontsize=20)
    ax[0].set_xlim(-200,200)
    ax[1].set_xlim(-200,200)
    
    fname = 'frame'+str(i).zfill(4)+'.png'
    fig.savefig(fname)
    
    return line,

anim = animation.FuncAnimation(fig, fplot, frames=101, repeat=False)
plt.close()
HTML(anim.to_jshtml())

In [ ]:
anim.save('two_stegoton_trains.mp4')

# Animation of solution values vs. Riemann invariants

In [ ]:
fig, ax = plt.subplots(figsize=[8,8], nrows=1, ncols=1)

ax.set_ylim(-0.5,0.5)
ax.set_xlim(0,0.8)
ax.set_xlabel(r'$\sigma$',fontsize=20)
ax.set_ylabel(r'$u$',fontsize=20);
line, = ax.plot([], [], lw=2)

xc = claw.frames[0].grid.x.centers

Kavg = 8/5.
rhoavg = 5/2
sigavg = np.exp(Kavg*q[0,:]) - 1.
uB = q[1,:] / rhoavg
ss = np.linspace(0,1)
uu = 1-np.sqrt(ss+1)

line3, = ax.plot(ss,uu,'-k')
line4, = ax.plot(ss,-uu,'-k')

def fplot(i):
    t = i*claw.tfinal/claw.num_output_times

    frame = claw.frames[i]
    q = frame.q
    xc = frame.grid.x.centers
    rho = frame.aux[0,:]
    u = q[1,:]/rho
    sig = stress(frame)
    
    sig_window = np.lib.stride_tricks.sliding_window_view(sig, cpl)
    u_window = np.lib.stride_tricks.sliding_window_view(u, cpl)
    
    sig_avg = sig_window.mean(1)
    u_avg = u_window.mean(1)

    line.set_data(sig_avg, u_avg)
    ax.set_title("t= {:.1f}".format(frame.t))
    
    return line,

anim = animation.FuncAnimation(fig, fplot, frames=401, repeat=False)
plt.close()
HTML(anim.to_jshtml())

In [ ]:
anim.save('riemann_invariants.mp4')

# Combined animation

In [ ]:
ax = plt.figure(layout="constrained",figsize=(10,6)).subplot_mosaic(
    """
    AAD
    CCD
    """
)
fig = plt.gcf()

ax['A'].set_ylim(-0.4,0.4)
ax['C'].set_ylim(-0.02,1.6)
line, = ax['A'].plot([], [], lw=2)
line2, = ax['C'].plot([], [], lw=2)
line3, = ax['D'].plot([], [], lw=2)

ax['D'].set_ylim(-0.5,0.5)
ax['D'].set_xlim(0,0.8)
ax['D'].set_xlabel(r'$\sigma$',fontsize=20)
ax['D'].set_ylabel(r'$u$',fontsize=20);

line4, = ax['D'].plot(ss,uu,'-k')
line5, = ax['D'].plot(ss,-uu,'-k')
xc = claw.frames[0].grid.x.centers

def fplot2(i):
    t = i*claw.tfinal/claw.num_output_times

    frame = claw.frames[i]
    eps = frame.q[0,:]
    rho = frame.aux[0,:]
    K = frame.aux[1,:]
    sigma = np.exp(K*eps)-1
    q2 = frame.q[1,:]
    u = q2/rho
    line.set_data(xc, -q2/rho)
    line2.set_data(xc, sigma)
    ax['A'].set_title("t= {:.1f}".format(frame.t))
    ax['A'].set_ylabel('u',fontsize=20)
    ax['C'].set_ylabel(r'$\sigma$',fontsize=20)
    ax['A'].set_xlim(-200,200)
    ax['C'].set_xlim(-200,200)

    sig_window = np.lib.stride_tricks.sliding_window_view(sigma, cpl)
    u_window = np.lib.stride_tricks.sliding_window_view(u, cpl)
    
    sig_avg = sig_window.mean(1)
    u_avg = u_window.mean(1)

    line3.set_data(sig_avg, u_avg)

    return line,

anim = animation.FuncAnimation(fig, fplot2, frames=401, repeat=False)
plt.close()
HTML(anim.to_jshtml())

In [ ]:
anim.save('combined.mp4')